<a href="https://colab.research.google.com/github/88FaHaD/ML/blob/main/mobilenetv3smallimageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-03-01 13:39:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-01 13:39:34 (59.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2024-03-01 13:39:38--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.207, 172.217.193.207, 172.217.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   158MB/s    in 1.0s    

2024-03-01 13:39:39 (158 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent

In [ ]:
# Create training and test directories
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [ ]:
import tensorflow as tf


# using image data directory

In [ ]:
import tensorflow as tf
IMG_SIZE = (224, 224) # define image size
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical", # what type are the labels?
                                                                            batch_size=32) # batch_size is 32 by default, this is generally a good number
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical")

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [ ]:
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[8.24553585e+00 7.24553585e+00 2.24553585e+00]
   [8.24553585e+00 8.24553585e+00 2.45535731e-01]
   [1.14598217e+01 1.14598217e+01 3.45982170e+00]
   ...
   [1.01151533e+01 1.14384623e+01 3.60011649e+00]
   [1.35449715e+01 1.40360432e+01 3.79050732e+00]
   [1.31578360e+01 1.36489077e+01 3.40337157e+00]]

  [[9.73660660e+00 8.73660660e+00 3.73660707e+00]
   [1.04205999e+01 1.04205999e+01 2.42059946e+00]
   [1.29508934e+01 1.29508934e+01 4.95089293e+00]
   ...
   [1.03125219e+01 1.13125219e+01 3.31252193e+00]
   [1.35267859e+01 1.48954277e+01 6.05262423e+00]
   [1.29062157e+01 1.56428223e+01 6.11603642e+00]]

  [[1.04553576e+01 9.00000000e+00 3.54464245e+00]
   [1.14553576e+01 1.07723217e+01 3.00000000e+00]
   [1.32764673e+01 1.25934305e+01 4.91868639e+00]
   ...
   [2.18225937e+01 2.16274605e+01 1.08411427e+01]
   [1.55793753e+01 1.69336720e+01 8.19545937e+00]
   [1.07578602e+01 1.43095722e+01 6.85421467e+00]]

  ...

  [[1.04227661e+02 7.18705215e+01 3.07079029e+01]
   [1

In [18]:
import tensorflow as tf

# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.MobileNetV3Small(include_top=False, weights='imagenet')

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

# 3. Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 5. Pass the inputs to the base_model (note: MobileNetV3 inputs don't have to be normalized)
x = base_model(inputs)
# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=["accuracy"])

# 10. Fit the model (we use less steps for validation so it's faster)
history_10_percent = model_0.fit(train_data_10_percent,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_10_percent),
                                 validation_data=test_data_10_percent,
                                 # Go through less of the validation data so epochs are faster (we want faster experiments!)
                                 validation_steps=int(0.25 * len(test_data_10_percent)),
                                 # Track our model's training logs for visualization later
                                 callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_feature_extract")])


Shape after base_model: (None, 7, 7, 576)
After GlobalAveragePooling2D(): (None, 576)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extract/20240301-150444
Epoch 1/5
24/24 [==============================] - 21s 672ms/step - loss: 2.4007 - accuracy: 0.1613 - val_loss: 1.9355 - val_accuracy: 0.3273
Epoch 2/5
24/24 [==============================] - 19s 791ms/step - loss: 1.7320 - accuracy: 0.4507 - val_loss: 1.4635 - val_accuracy: 0.5724
Epoch 3/5
24/24 [==============================] - 18s 741ms/step - loss: 1.3320 - accuracy: 0.6307 - val_loss: 1.1959 - val_accuracy: 0.6645
Epoch 4/5
24/24 [==============================] - 17s 658ms/step - loss: 1.1107 - accuracy: 0.7027 - val_loss: 1.0488 - val_accuracy: 0.7204
Epoch 5/5
24/24 [==============================] - 19s 791ms/step - loss: 0.9693 - accuracy: 0.7387 - val_loss: 0.9593 - val_accuracy: 0.7155


In [19]:
base_model.summary()



Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling_6 (Rescaling)     (None, None, None, 3)        0         ['input_7[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, None, None, 16)       432       ['rescaling_6[0][0]']         
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, None, None, 16)       64        ['Conv[0][0]']                
 lization)                                                                         